In [1]:
import pandas as pd, numpy as np
import time

In [2]:
# from joblib import Parallel, delayed  
# import multiprocessing
# num_cores = multiprocessing.cpu_count()
# results = Parallel(n_jobs=num_cores)(delayed(getOccurences)(3, 10) for i in sims)
# print num_cores

In [3]:
# movements are east or west, and north or south. For east-west movements, the indicator is either +/-x resp,
# that for north-south is +/- y respectively. The 
def getOccurences(blocks, moves=10):    
    x = 0 # initializing x variable
    y = 0 # initializing y variable
    draws = np.random.uniform(0,1,moves)
    x += len(draws[(draws <= 0.25)])
    x -= len(draws[(draws > 0.25) & (draws <= 0.5)])
    y += len(draws[(draws > 0.5) & (draws <= 0.75)])
    y -= len(draws[(draws > .75) &(draws <= 1.0)])
    if (abs(x) >= blocks) | (abs(y) >= blocks):
        return 1
    return 0

def runSimulation(blocks = 3, moves = 10, sims = 1000):
    runs = np.repeat(blocks,sims)
    ds = pd.DataFrame(runs)
    ds['wins'] = ds[0].apply(getOccurences, args = (moves,))
    return ds['wins'].mean()

# Another version of function that generates a square matrix and loops over each cell using apply(map)
def getProb(blocks, n):
    cols = np.repeat(blocks,n)
    mat  = np.tile(cols, (n,1))
    dat = pd.DataFrame(mat)
    ndat = dat.applymap(getOccurences)
    return ndat.sum().sum()*1.0/(n*n)
def getResult(blocks=3, moves = 10, sims = 1000):
    result = [runSimulation(blocks,moves,999) for i in range(sims)]
    return np.round(np.mean(result), 11)

In [4]:
# test1 = getProb(3, 1000)
# t = 0 # number of occurences
# sims = 1000000 # number of simulations
# start = time.time()
# for sim in range(sims):
#     t += getOccurences(3, 10) 
# end = time.time()
# print t*1.0/sims
# print end-start

In [12]:
print 'At-least 3 blocks away after 10 moves, Prob = %.10f' %getResult(3,10, 1007)
print 'At-least 10 blocks away after 60 moves, Prob = %.10f' %getResult(10,60, 1007)

At-least 3 blocks away after 10 moves, Prob = 0.4550069513
At-least 10 blocks away after 60 moves, Prob = 0.1579860973


In [6]:
def reachesPoint(blocks, moves):    
    x = 0 # initializing x variable
    y = 0 # initializing y variable
    draws = np.random.uniform(0,1,moves)
    for p in draws:
        if p <= 0.25:
            x+=1
            if abs(x) >= blocks:
                return 1
        elif (p > 0.25) & (p <= 0.5):
            x-=1
            if abs(x) >= blocks:
                return 1
        elif (p > 0.5) & (p <= 0.75):
            y += 1
            if abs(y) >= blocks:
                return 1
        else:
            y -= 1
            if abs(y) >= blocks:
                return 1
    return 0

def runSimulation2(blocks , moves , sims):
    runs = np.repeat(blocks,sims)
    ds = pd.DataFrame(runs)
    ds['wins'] = ds[0].apply(reachesPoint, args = (moves,))
    return ds['wins'].mean()

def getResult2(blocks=5, moves = 10, sims = 1000):
    result = [runSimulation2(blocks,moves,999) for i in range(sims)]
    return np.mean(result)

In [7]:
print 'Ever at-least 5 blocks away after 10 moves, Prob = %.10f' %getResult2(5,10, 1000)
print 'Ever at-least 10 blocks away after 60 moves, Prob = %.10f' %getResult2(10,60, 1000)

Ever at-least 5 blocks away after 10 moves, Prob = 0.1054844845
Ever at-least 10 blocks away after 60 moves, Prob = 0.2571901902


In [8]:
def occurencesAfterReachesPoint(blocks, moves):    
    x = 0 # initializing x variable
    y = 0 # initializing y variable
    indicator = 0
    draws = np.random.uniform(0,1,moves)
    for p in draws:
        if (x > 1) & (indicator == 0):
            indicator = 1
        if p <= 0.25:
            x+=1
        elif (p > 0.25) & (p <= 0.5):
            x-=1
    if (x < -1*blocks) & (indicator == 1):
                return 1
    return 0

def runSimulation3(blocks , moves , sims):
    runs = np.repeat(blocks,sims)
    ds = pd.DataFrame(runs)
    ds['wins'] = ds[0].apply(occurencesAfterReachesPoint, args = (moves,))
    return ds['wins'].mean()

def getResult3(blocks=5, moves = 10, sims = 1000):
    result = [runSimulation3(blocks,moves,999) for i in range(sims)]
    return np.mean(result)

In [9]:
print 'west of 1st West after east of 1st East away in 10 moves, Prob = %.10f' %getResult3(1,10, 1000)
print 'west of 1st West after east of 1st East away in 30 moves, Prob = %.10f' %getResult3(1,30, 1000)

west of 1st West after east of 1st East away in 10 moves, Prob = 0.0059689690
west of 1st West after east of 1st East away in 30 moves, Prob = 0.0776186186


What is the average number of moves until the first time the tourist is at least 10 city blocks (as the crow flies) from Broadway and Broadway.

In [10]:
def movesBefore(blocks):    
    x = 0 # initializing x variable
    y = 0 # initializing y variable
    draws = np.random.uniform(0,1,1000)
    counter = 0
    for p in draws:
        counter += 1
        if p <= 0.25:
            x+=1
            if abs(x) >= blocks:
                return counter
        elif (p > 0.25) & (p <= 0.5):
            x-=1
            if abs(x) >= blocks:
                return counter
        elif (p > 0.5) & (p <= 0.75):
            y += 1
            if abs(y) >= blocks:
                return counter
        else:
            y -= 1
            if abs(y) >= blocks:
                return counter
    return -1

def runSimulation4(blocks , sims):
    runs = np.repeat(blocks,sims)
    ds = pd.DataFrame(runs)
    res = np.array(ds[0].apply(movesBefore))
    counts = res[res>0]
    return np.mean(counts)

def getResult4(blocks=5, sims = 1000):
    result = [runSimulation4(blocks, 999) for i in range(sims)]
    return np.mean(result)

In [13]:
print 'Tourist at at-least 10 blocks away in %.10f moves on average' %getResult4(10, 1099)
print 'Tourist at at-least 60 blocks away in %.10f moves on average' %getResult4(60, 1099)

Tourist at at-least 10 blocks away in 117.6608639197 moves on average
Tourist at at-least 60 blocks away in 823.5127622682 moves on average
